In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-13 15:19:24,828 utils 400 [INFO]    [logger_func] start 


In [184]:
# Data Load
is_clf_out = [True, False][0]
is_no_out_flg = [True, False][1]
is_rm_out = [True, False][0]
#========================================================================
# Base Model Load
base = utils.read_df_pkl('../input/base_no_out_clf.gz').set_index(key)
ens_list = glob.glob('../ensemble/lgb_ensemble/*.gz')
# ens_list = ['../stack/0213_152_stack_ridge_set-all_lgb27_NN0_other0_OUT29.7052_CV3.616445337563796_LB.gz']
ens_list = ['../stack/0214_074_stack_ridge_set-all_lgb20_NN0_other0_OUT29.7295_CV3.618401419254512_LB.gz']
# ens_list = ['../model/LB3665_set/0213_152_stack_ridge_set-all_lgb27_NN0_other0_OUT29.7052_CV3.616445337563796_LB.gz']
for path in ens_list:
    try:
        cv = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
    except AttributeError:
        print(path)
        cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
    
#     try:
#         blend = utils.read_df_pkl(path).set_index(key)['pred_mean']
#     except KeyError:
#         blend = utils.read_df_pkl(path).set_index(key)['prediction']
    blend = utils.read_df_pkl(path)[[key, 'prediction']].set_index(key)
    base[f'base_{cv}'] = blend
    base_cols = [col for col in base.columns if col.count('base_')]
#========================================================================

#========================================================================
# Clf Out Model
if is_clf_out:
    out_list = glob.glob('../clf_min_thres_ensemble/*.gz')
    out_list = ['../model/LB3665_set/0213_151_stack_ridge_set-all_lgb9_NN0_other0_OUT33.1624_CV2.1661770689682887_LB.gz']
    for path in out_list:
        cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        try:
            blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
        base[f"clf_out_{cv}"] = blend
    clf_out_cols = [col for col in base.columns if col.count('clf_out_2')]
#========================================================================

#========================================================================
# No Out Flg Model
if is_no_out_flg:
    no_out_flg_list = glob.glob('../no_out_flg_ensemble/*.gz')
    no_out_flg_list = ['../stack/']

    for path in no_out_flg_list:
        cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        try:
            blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
        base[f"no_out_flg_{cv}"] = blend
    no_out_cols = [col for col in base.columns if col.count('no_out_flg_1')]
#========================================================================

#========================================================================
# No Out Model
if is_rm_out:
    rm_out_list = ['../model/LB3665_set/0213_145_stack_ridge_set-rm_out_lgb31_NN0_other0_OUT0_CV1.546157292555354_LB.csv.gz']
    rm_out_list += glob.glob('../ensemble/NN_ensemble/*.gz')

    for path in rm_out_list:
        if path.count('NN') and path.count('all'):continue
        try:
            cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        except AttributeError:
            print(path)
            sys.exit()
        try:
            blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
        if path.count('lgb'):
            col = f"lgb_rm_out_{cv}"
        elif path.count('NN'):
            col = f"nn_rm_out_{cv}"
        base[col] = blend
        rm_out_cols = [col for col in base.columns if col.count('rm_out')]
#========================================================================


#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# clf['clf_pred'] =  clf['pred_mean']
#========================================================================

#========================================================================
# Indexをそろえる
base['clf_pred'] = clf['clf_pred']
pred_cols = [col for col in base.columns if col.count('-')]
ignore_list += [col for col in base.columns if col.count('hist') or col.count('new_') or col.count('month')]
pred_cols = sorted(pred_cols)
base = base[[target, 'clf_pred', 'no_out_flg'] + pred_cols]
# base = base[[target, 'clf_pred', 'no_out_flg'] + base_cols]
base.head()
#========================================================================

100%|██████████| 1/1 [00:00<00:00,  8.89it/s]


,target,clf_pred,no_out_flg,base_3-618401419254512,clf_out_2-1661770689682887_LB,lgb_rm_out_1-546157292555354_LB-csv,nn_rm_out_1-5559552751406402,nn_rm_out_1-5563906165871186,nn_rm_out_1-5594050847862546,nn_rm_out_1-560574574717595,nn_rm_out_1-5613126708390401,nn_rm_out_1-5620922839013212,nn_rm_out_1-5653863332859428
card_id,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,0.000444,1.0,-0.318154,-0.308417,-0.329140,-0.393493,-0.239650,-0.333955,-0.239385,-0.410733,-0.510899,-0.284880
C_ID_3d0044924f,0.392913,0.007831,0.0,-0.419848,0.033420,0.315994,0.204009,0.269381,0.623660,-0.078358,0.162143,0.220517,0.086854
C_ID_d639edf6cd,0.688056,0.004074,0.0,0.720410,0.439877,0.744477,0.587247,0.542163,0.504456,0.451000,0.554720,0.939229,0.153362
C_ID_186d6a6901,0.142495,0.000797,0.0,0.189931,0.185103,0.193574,0.264077,0.214717,0.246618,0.646056,0.485820,0.283804,0.298219
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0,-0.059109,-0.190333,-0.224593,-0.413504,-0.186307,-0.177461,-0.139813,-0.354343,-0.092744,0.181141


In [187]:
nn_cols = [col for col in base.columns if col.count('nn_')]
base['nn_rm_out_pred'] = base[nn_cols].mean(axis=1)
base.drop(nn_cols, axis=1, inplace=True)
rm_out_cols = [col for col in base.columns if col.count('rm_out')]
rm_out_cols
base.head()

,target,clf_pred,no_out_flg,base_3-618401419254512,clf_out_2-1661770689682887_LB,lgb_rm_out_1-546157292555354_LB-csv,nn_rm_out_pred
card_id,,,,,,,
C_ID_92a2005557,-0.820283,0.000444,1.0,-0.318154,-0.308417,-0.329140,-0.344713
C_ID_3d0044924f,0.392913,0.007831,0.0,-0.419848,0.033420,0.315994,0.212601
C_ID_d639edf6cd,0.688056,0.004074,0.0,0.720410,0.439877,0.744477,0.533168
C_ID_186d6a6901,0.142495,0.000797,0.0,0.189931,0.185103,0.193574,0.348473
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0,-0.059109,-0.190333,-0.224593,-0.169004


In [189]:
#========================================================================
# Simple Blender 
is_blend = [True, False][0]
pred = 'prediction'
if is_blend:
    min_thres = 0.01
    # out_ratio_1 = 0.26
    # out_ratio_2 = 0.74
    
    #========================================================================
    # Clf Out 
    base[pred] = base[base_cols]
    # base.loc[base['clf_pred']< min_thres, pred] = base.loc[base['clf_pred']< min_thres, out_cols[1]]*out_ratio_1 + base.loc[base['clf_pred']< min_thres, out_cols[2]]*out_ratio_2
    base.loc[base['clf_pred']< 0.01, pred] = base.loc[base['clf_pred']< 0.01, clf_out_cols[0]]
    # base.loc[base['clf_pred']< min_thres, pred] = base.loc[base['clf_pred']< 0.01, out_cols[0]]
    # + base.loc[base['clf_pred']< min_thres, out_cols[2]]*out_ratio_2
    
    #========================================================================
    # Remove Out
    rm_out_pred = 'rm_out_pred'
    base[rm_out_pred] = base[rm_out_cols[0]]*0.8 + base[rm_out_cols[1]]*0.2
    base.loc[base['clf_pred']<min_thres, pred] = base.loc[base['clf_pred']<min_thres, rm_out_pred]
    base.loc[base['no_out_flg']==1, pred] = base.loc[base['no_out_flg']==1, rm_out_pred]
    base.loc[base[pred].isnull(), pred] = base.loc[base[pred].isnull(), clf_out_cols[0]]
    #========================================================================
    
    #========================================================================
    # Manual Pred
    # base.loc[base['clf_pred']>0.471, pred] = -9.12
    # base.loc[base['clf_pred']>0.511, pred] = -25.32
    # base.loc[base['no_out_flg']==1, pred] = base.loc[base['no_out_flg']==1, no_out_cols[0]]
    #========================================================================
else:
    base[pred] = base[pred_cols].mean(axis=1)
    out_cols = []
    min_thres = 0

train = base[~base[target].isnull()]
y_pred = train[pred].values
y_train = train[target].values
score = np.sqrt(mean_squared_error(y_train, y_pred))
print(score)

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend_base{len(base_cols)}_clf_out{len(clf_out_cols)}_rm_out{len(rm_out_cols)}_no_out_blend0_minthres{min_thres}_CV{score}_LB')
# submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_MANUAL_2level_clf0.511_clf0.471_processing_blend{len(base_cols)}_out_blend{len(out_cols)-2}_no_out_blend{len(no_out_cols)-1}_minthres{min_thres}_CV{score}_LB')
display(submit.head())
#========================================================================

3.6090493568345723


,target
card_id,
C_ID_0ab67a22ab,-1.464253
C_ID_130fd0cbdd,-0.128765
C_ID_b709037bc5,-1.636066
C_ID_d27d835a9f,-0.017996
C_ID_2b5e3df5c2,-1.215995


In [188]:
# Remove Out
train = base[~base[target].isnull()]
# y_train = train[train[target]>-30][train['clf_pred']<0.01][target].values
# y_pred = train[train[target]>-30][train['clf_pred']<0.01]['clf_out_2-1661770689682887_LB'].values

y_train = train[train[target]>-30][target].values
y_pred = train[train[target]>-30]['nn_rm_out_pred'].values

# # Clf Out
# y_train = train[train['clf_pred']<0.01][target].values
# y_pred = train[train['clf_pred']<0.01]['clf_out_2-1661770689682887_LB'].values

# y_train = train[train['clf_pred']<0.01][target].values
# y_pred = train[train['clf_pred']<0.01]['base_3-616445337563796_LB'].values

# y_train = train[target].values
# y_pred = train['base_3-618401419254512'].values

score = np.sqrt(mean_squared_error(y_train, y_pred))
print(score)

1.5396981797466824


In [98]:
# Save Stack
utils.to_pkl_gzip(obj=base[pred].reset_index(), path= f"../ensemble/lgb_ensemble/{start_time[4:12]}_blend_lgb25_CV3-627555_LB3.675")

In [46]:
utils.to_pkl_gzip(obj=base['prediction'].reset_index(), path=f'../ensemble/good_submit_ensemble/{start_time[4:12]}_submit_base-stack-lgb8_rm_out-lgb1_no_out-0_clf-thres{min_thres}_CV{score}_LB3.667')

In [36]:
clf = utils.read_pkl_gzip('../input/base_clf.gz')
clf = clf[clf['clf_pred']<0.01]
kfold = utils.read_pkl_gzip('../input/ods_NoOut_kfold.gz')
row = 0
for fold_no, (trn_idx, val_idx) in enumerate(zip(*kfold)):
#     row += clf.loc[clf[key].isin(val_idx), :].shape[0]
print(row)

c


SystemExit: 

In [31]:
#========================================================================
# Manual PostProcessing
# Setting
# upper_1. Best Score: 3.6114026721733525 | Best Manual: 25.319999999999993 | Max Thres: 0.511
# upper_2. Best Score: 3.6105940647875325 | Best Manual: 9.119999999999997 | Max Thres: 0.471
#========================================================================

#========================================================================
# Base Model Pred
base = utils.read_df_pkl('../input/base_no_out_clf.gz').set_index(key)
best_model = utils.read_pkl_gzip('../ensemble/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz')[[key, 'pred_mean']].set_index(key)
base['pred_mean'] = best_model['pred_mean']
#========================================================================

#========================================================================
# No Outlier Model
# out_list = ['../out_ensemble/0208_215_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz']
# out_model = utils.read_pkl_gzip('../clf_min_thres_ensemble/0211_140_elo_NN_stack_E4_linear1_290feat_const2_lr0.001_batch128_epoch30_CV1.4159121483888761.gz')[[key, 'prediction']].set_index(key)
# out_model = utils.read_pkl_gzip(out_list[0])[[key, 'prediction']].set_index(key)
# base['no_out_pred'] = out_model['prediction']
#========================================================================

#========================================================================
# No Outlier Model の予測の欠損はAll Modelで埋める
# base.loc[base['no_out_pred'].isnull(), 'no_out_pred'] = base.loc[base['no_out_pred'].isnull(), 'pred_mean']
# base.loc[base['no_out_pred'].isnull(), 'no_out_pred'] = 0
#========================================================================

#========================================================================
# Classifier
# clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf_3 = utils.read_pkl_gzip('../stack/0211_163_outlier_classify_9seed_lgb_binary_CV0-043388680571356614_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf['pred_mean_2'] = clf_2['pred_mean']
# clf['pred_mean_3'] = clf_3['pred_mean']
# clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# clf = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key)
# base['clf_pred'] = clf['clf_pred']
# base['clf_pred'] = clf['pred_mean']
#========================================================================

is_manual = [True, False][0]
score_list = []
ratio_list = []
min_score = 100
pred = 'prediction'

train = base[~base[target].isnull()]
y_train = train[target].values
train[pred] = 0
# train[pred] = train['no_out_pred']
min_thres = 0.0072
upper_thres = 0.511
upper_value = -25.32
# for min_thres in np.arange(0.005, 0.01, 0.0002):
# for min_thres in [0.01]:
for max_thres in np.arange(0.5, 0.52, 0.001):
    
#     length = train.loc[train['clf_pred']> min_thres].shape[0]
    
    # Under PostProcessing
#     train.loc[train['clf_pred']< min_thres, pred] = train.loc[train['clf_pred']< min_thres, 'no_out_pred']
#     train.loc[train['clf_pred']>=min_thres, pred] = train.loc[train['clf_pred']>=min_thres, 'pred_mean']
#     train.loc[train['no_out_flg']==1, pred] =  train.loc[train['no_out_flg']==1, 'no_out_pred']

    # 閾値以上のtargetをマニュアル設定する
    if is_manual:
        tmp_pred = train[pred].values
        for manual in np.arange(24.0, 26, 0.2):
            train[pred] = tmp_pred.copy()
            train.loc[train['clf_pred']> max_thres, pred] = manual*-1
            # Upper PosrProcessing
            train.loc[train['clf_pred']> upper_thres, pred] = upper_value
    
            y_pred = train[pred].values
        
            score = np.sqrt(mean_squared_error(y_train, y_pred))
            if score<min_score:
                min_score = score
                best_thres = max_thres
                best_value = manual
                score_list.append(min_score)
                ratio_list.append(best_thres)
            if score<3.611:
#                 print(f"Best Score: {min_score} | Min Thres: {best_thres} | Max Thres: {max_thres} | MANUAL: {-1*manual}")
                print(f"Score: {score} | Best Score: {min_score} | Best Manual: {best_value} | Max Thres: {best_thres}")
        
    else:
        y_pred = train[pred].values
    
        score = np.sqrt(mean_squared_error(y_train, y_pred))
        if score<min_score:
            min_score = score
            best_thres = min_thres
            score_list.append(min_score)
            ratio_list.append(best_thres)
        print(f"Best Score: {min_score} | Best Thres: {best_thres} | LENGTH: {length}")

100%|██████████| 3/3 [00:00<00:00, 65.29it/s]


0.01


SystemExit: 

In [15]:
blend_num = 3
out_blend_num = 2
thres = 0.0072
base.loc[base['clf_pred']< thres, pred] = base.loc[base['clf_pred']< thres, 'no_out_pred']
base.loc[base['clf_pred']>=thres, pred] = base.loc[base['clf_pred']>=thres, 'pred_mean']
base.loc[base['no_out_flg']==1, pred] =  base.loc[base['no_out_flg']==1, 'no_out_pred']
display(base.head())

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend{blend_num}_0.4_0.3_0.3_out_blend{out_blend_num}_0.7_0.3_CV{min_score}_LB')
display(submit.head())

,target,no_out_flg,pred_mean,p1,p2,p3,no_out_pred,clf_pred,prediction
card_id,,,,,,,,,
C_ID_92a2005557,-0.820283,NaN,-0.312293,-0.277248,-0.299641,-0.295437,-0.339001,0.000444,-0.339001
C_ID_3d0044924f,0.392913,NaN,-0.402629,-0.464311,-0.299189,-0.659761,0.276268,0.007831,-0.402629
C_ID_d639edf6cd,0.688056,NaN,0.740270,0.764349,0.759806,0.823495,0.770552,0.004074,0.770552
C_ID_186d6a6901,0.142495,NaN,0.134626,0.101346,0.134031,0.057771,0.170184,0.000797,0.170184
C_ID_cdbd2c0db2,-0.159749,1.0,-0.184077,-0.222948,-0.242942,-0.158849,-0.238870,0.000251,-0.238870


,target
card_id,
C_ID_0ab67a22ab,-1.659109
C_ID_130fd0cbdd,-0.203953
C_ID_b709037bc5,-1.298058
C_ID_d27d835a9f,-0.052482
C_ID_2b5e3df5c2,-1.371860


In [ ]:
from itertools import product, permutations
#========================================================================
# Randomness Blendのスコアを見る
#========================================================================
score_list = []
best_ratio_list = []
min_score = 100
train = base[~base[target].isnull()]
y_train = train[target].values


cnt = 0
while cnt!=10000:
    cnt+=1
    np.random.seed(cnt)
    ratio_list = np.arange(0.05, 1.0, 0.05)
    np.random.seed(cnt+1); np.random.shuffle(ratio_list)
    r1 = ratio_list.copy()
    np.random.seed(cnt*2); np.random.shuffle(ratio_list)
    r2 = ratio_list.copy()
    np.random.seed(cnt*3); np.random.shuffle(ratio_list)
    r3 = ratio_list.copy()
    np.random.seed(cnt*4); np.random.shuffle(ratio_list)
    r4 = ratio_list.copy()
    ratio_combi = zip(r1, r2, r3, r4)
    
    for i, ratio in enumerate(ratio_combi):
        
        p1 = train[base_cols[0]].values
        p2 = train[base_cols[1]].values
        p3 = train[base_cols[2]].values
        p4 = train[base_cols[3]].values
#         if cnt==1:
#             print(
#                 np.sqrt(mean_squared_error(y_train, p1))
#                 ,np.sqrt(mean_squared_error(y_train, p2))
#                 ,np.sqrt(mean_squared_error(y_train, p3))
#                 ,np.sqrt(mean_squared_error(y_train, p4))
#             )
#             sys.exit()
        
        ratio = np.array(ratio)
        r_sum = ratio.sum()
        ratio /= r_sum
        a = ratio[0]
        b = ratio[1]
        c = ratio[2]
        d = ratio[3]
        coef_list = [a,b,c,d]
        
        combi_list = permutations(coef_list)
        for combi in combi_list:
            e = combi[0]
            f = combi[1]
            g = combi[2]
            h = combi[3]
            y_pred = p1*e + p2*f + p3*g + p4*h
            
            score = np.sqrt(mean_squared_error(y_train, y_pred))
            if score<min_score:
                min_score = score
                min_ratio = combi
                
                score_list.append(min_score)
                best_ratio_list.append(min_ratio)
        
    if cnt%200==0:
        print(f"BEST: {np.min(score_list)} | BEST RATIO: {best_ratio_list[np.argmin(score_list)]}")
        
print("Complete!!")

In [225]:
#========================================================================
# CV CHECKER 
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]
base_idx = train.reset_index()[[key, target]]

check_model_path = '../stack/0208_002_stack_lgb_lr0.01_235feats_multi157_valterm_5seed_4leaves_iter1031_TERM24_CV3-631440687534257_LB.gz'
check_model = utils.read_pkl_gzip(check_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
train = train.join(check_model)

y_train = train[target].values
check_pred = train['pred_mean'].values
check_score = np.sqrt(mean_squared_error(y_train, check_pred))
print(check_score)
#========================================================================


100%|██████████| 3/3 [00:00<00:00, 56.44it/s]

3.7031587732980125


In [ ]:
#========================================================================
# Corr Checker
from itertools import combinations
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key).query('no_out_flg==1')
ens_list = glob.glob('../ensemble/*.gz')
ens_list = glob.glob('../no_out_flg_ensemble/*.gz')
combi_path = combinations(ens_list, 2)

#=======================================================================
# Classifier
# clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf['pred_mean_2'] = clf_2['pred_mean']
# clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# base['clf_pred'] = clf['clf_pred']
# utils.to_pkl_gzip(obj=base.reset_index(), path='../input/base_clf')
# sys.exit()
#========================================================================

#========================================================================
# 相関
for p_1, p_2 in combi_path:
    try:
        m1 = utils.read_pkl_gzip(p_1)[[key, 'pred_mean']].set_index(key)
        base['p1'] = m1['pred_mean']
    except KeyError:
        m1 = utils.read_pkl_gzip(p_1)[[key, 'prediction']].set_index(key)
        base['p1'] = m1['prediction']
    try:
        m2 = utils.read_pkl_gzip(p_2)[[key, 'pred_mean']].set_index(key)
        base['p2'] = m2['pred_mean']
    except KeyError:
        m2 = utils.read_pkl_gzip(p_2)[[key, 'prediction']].set_index(key)
        base['p2'] = m2['prediction']
    if p_1.count('LB'):
        cv1 = re.search(r'CV([^/.]*)_LB.gz', p_1).group(1)
    else:
        cv1 = re.search(r'CV([^/.]*)', p_1.replace('.', '-')).group(1)
    if p_2.count('LB'):
        cv2 = re.search(r'CV([^/.]*)_LB.gz', p_2).group(1)
    else:
        cv2 = re.search(r'CV([^/.]*)', p_2.replace('.', '-')).group(1)
        
    corr = np.corrcoef(base['p1'], base['p2'].values).min()
    logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
#========================================================================